### Zones Instance Resizing

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

def resize_image(image_path, output_path, new_size):
    with Image.open(image_path) as img:
        img = img.convert('RGBA')  # Convert to RGBA if not already to ensure alpha is preserved
        resized_img = img.resize(new_size, Image.LANCZOS)  # Resize using LANCZOS filter for best quality
        resized_img.save(output_path, 'PNG')  # Save as PNG

def resize_images_in_directory(source_directory, sizes):
    base_output_directory = os.path.dirname(source_directory)
    size_to_output_directory = {size: os.path.join(base_output_directory, str(size)) for size in sizes}

    # Create all output directories
    for directory in size_to_output_directory.values():
        os.makedirs(directory, exist_ok=True)

    # Gather all PNG images
    images = [f for f in os.listdir(source_directory) if f.lower().endswith('.png')]
    
    # Resize image for each size and save in the corresponding directory
    for size in sizes:
        output_directory = size_to_output_directory[size]
        progress_bar = tqdm(images, desc=f'Resizing images to {size}x{size}', unit='img')
        for filename in progress_bar:
            file_path = os.path.join(source_directory, filename)
            output_path = os.path.join(output_directory, filename)
            
            # Check if the output file already exists
            resize_image(file_path, output_path, (size, size))
            if not os.path.exists(output_path):
                resize_image(file_path, output_path, (size, size))

# Example usage:
source_directory = "/Users/laeh/Desktop/art0/assets/zones/light/v0/5040"
sizes = [3360]

# # Example usage:
# source_directory = "/Users/laeh/Desktop/art0/assets/zones/louvre/v0/5040"
# sizes = [3360,1680]

# resize_images_in_directory(source_directory, sizes)



## zones2video + Intro

In [ ]:
import os
import random
import subprocess
import shutil
from collections import defaultdict
from PIL import Image
import uuid
from concurrent.futures import ThreadPoolExecutor

def sample_unique_image_sets(source_directory, categories, num_sets):
    images_by_category = defaultdict(list)
    for filename in os.listdir(source_directory):
        for category in categories:
            if category in filename:
                images_by_category[category].append(os.path.join(source_directory, filename))

    for category, images in images_by_category.items():
        if len(images) < num_sets:
            raise ValueError(f"Not enough images for category '{category}'. Required: {num_sets}, Available: {len(images)}")

    preselected_images_by_category = {category: random.sample(images, num_sets) for category, images in images_by_category.items()}

    set_image_paths = []
    for i in range(num_sets):
        for category in categories:
            selected_image = preselected_images_by_category[category][i]
            set_image_paths.append(selected_image)

    return set_image_paths

def create_frame(args):
    background_image, frame_num, total_frames, fade_frames, images_with_masks, frames_directory = args
    current_frame = background_image.copy()
    current_image_index = (frame_num - 1) // fade_frames
    current_image_phase = (frame_num - 1) % fade_frames

    for i, (image, mask) in enumerate(images_with_masks):
        if i <= current_image_index:
            opacity = int(255 * current_image_phase / fade_frames) if i == current_image_index else 255
            fading_mask = mask.point(lambda p: min(p, opacity))
            fading_image = image.copy()
            fading_image.putalpha(fading_mask)
            current_frame = Image.alpha_composite(current_frame, fading_image)

    frame_path = os.path.join(frames_directory, f"frame_{frame_num:05}.png")
    current_frame.save(frame_path)
    print(f"Processed frame {frame_num}/{total_frames}")

def create_sequential_fade_animation_with_background(background_path, image_paths, output_video_path, fade_frames=100):
    short_uid = uuid.uuid4().hex[:4]  # Generate a shorter unique identifier
    frames_directory = '/Users/laeh/Pictures/.tmp_frames-'+short_uid
    if not os.path.exists(frames_directory):
        os.makedirs(frames_directory)

    background_image = Image.open(background_path).convert('RGBA')
    images_with_masks = [(Image.open(path).convert('RGBA'), Image.open(path).convert('RGBA').split()[3]) for path in image_paths]
    total_images = len(images_with_masks)
    total_frames = fade_frames * total_images

    args_list = [(background_image, frame_num, total_frames, fade_frames, images_with_masks, frames_directory) for frame_num in range(1, total_frames + 1)]
    with ThreadPoolExecutor(max_workers=16) as executor:  # Adjust max_workers as needed
        executor.map(create_frame, args_list)

    return frames_directory

def create_video_from_frames(frames_directory, output_video_path, framerate=60):
    command = [
        'ffmpeg',
        '-framerate', str(framerate),
        '-i', os.path.join(frames_directory, 'frame_%05d.png'),
        '-c:v', 'libx264',
        '-profile:v', 'high',
        '-crf', '20',
        '-pix_fmt', 'yuv420p',
        output_video_path
    ]
    subprocess.run(command, check=True)

def remove_frames_directory(frames_directory):
    shutil.rmtree(frames_directory)
    print(f"Removed frames directory: {frames_directory}")

def create_art0_video(background_path, source_directory, num_sets, fade_frames=100):
    categories = ["pink", "green", "cyan", "red", "yellow", "orange", "blue", "indigo"]
    base_name = os.path.basename(os.path.normpath(source_directory))
    short_uid = uuid.uuid4().hex[:4]  # Generate a shorter unique identifier
    results_directory = "/Users/laeh/Desktop/art0/results/MacStudio"
    output_video_path = os.path.join(results_directory, f"{base_name}-{num_sets}sets-{short_uid}.mp4")

    try:
        set_image_paths = sample_unique_image_sets(source_directory, categories, num_sets)
        frames_directory = create_sequential_fade_animation_with_background(background_path, set_image_paths, output_video_path, fade_frames)
        create_video_from_frames(frames_directory, output_video_path, 60)
        print(f"Video created successfully: {output_video_path}")
        remove_frames_directory(frames_directory)
    except ValueError as e:
        print(e)
    except subprocess.CalledProcessError as e:
        print(f"Failed to create video due to an error: {e}")

if __name__ == "__main__":
    num_sets = 2
    source_directory = "./assets/zones/light/v2/1680"
    background_path = "./assets/intro/white@1680.png"
    create_art0_video(background_path, source_directory, num_sets)



## zones2video + Intro + Outro

In [ ]:
import os
import random
import subprocess
import shutil
from collections import defaultdict
from PIL import Image
import uuid
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm  # Import tqdm for the progress bar

def sample_unique_image_sets(source_directory, categories, num_sets):
    images_by_category = defaultdict(list)
    for filename in os.listdir(source_directory):
        for category in categories:
            if category in filename:
                images_by_category[category].append(os.path.join(source_directory, filename))

    for category, images in images_by_category.items():
        if len(images) < num_sets:
            raise ValueError(f"Not enough images for category '{category}'. Required: {num_sets}, Available: {len(images)}")

    preselected_images_by_category = {category: random.sample(images, num_sets) for category, images in images_by_category.items()}

    set_image_paths = []
    for i in range(num_sets):
        for category in categories:
            selected_image = preselected_images_by_category[category][i]
            set_image_paths.append(selected_image)

    return set_image_paths

def create_frame(args):
    background_image, frame_num, total_frames, fade_frames, images_with_masks, frames_directory = args
    current_frame = background_image.copy()
    current_image_index = (frame_num - 1) // fade_frames
    current_image_phase = (frame_num - 1) % fade_frames

    for i, (image, mask) in enumerate(images_with_masks):
        if i <= current_image_index:
            opacity = int(255 * current_image_phase / fade_frames) if i == current_image_index else 255
            fading_mask = mask.point(lambda p: min(p, opacity))
            fading_image = image.copy()
            fading_image.putalpha(fading_mask)
            current_frame = Image.alpha_composite(current_frame, fading_image)

    frame_path = os.path.join(frames_directory, f"frame_{frame_num:05}.png")
    current_frame.save(frame_path)

def create_sequential_fade_animation_with_background(background_path, image_paths, frames_directory, fade_frames=100):
    background_image = Image.open(background_path).convert('RGBA')
    images_with_masks = [(Image.open(path).convert('RGBA'), Image.open(path).convert('RGBA').split()[3]) for path in image_paths]
    total_images = len(images_with_masks)
    total_frames = fade_frames * total_images

    args_list = [(background_image, frame_num, total_frames, fade_frames, images_with_masks, frames_directory) for frame_num in range(1, total_frames + 1)]
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(create_frame, args_list), total=total_frames, desc="Generating animation frames"))

    return total_frames

def add_outro_frames(frames_directory, outro_image_path, start_frame_num, fade_frames=200):
    outro_image = Image.open(outro_image_path).convert("RGBA")
    for i in tqdm(range(1, fade_frames + 1), desc="Adding outro frames"):
        opacity = int(255 * (i / fade_frames))
        outro_frame = Image.new("RGBA", outro_image.size, (255, 255, 255, 0))
        outro_frame.paste(outro_image, (0, 0), outro_image.split()[3].point(lambda _: opacity))
        
        frame_path = os.path.join(frames_directory, f"frame_{start_frame_num + i:05}.png")
        outro_frame.save(frame_path)

def create_video_from_frames(frames_directory, output_video_path, framerate=60):
    command = [
        'ffmpeg',
        '-framerate', str(framerate),
        '-i', os.path.join(frames_directory, 'frame_%05d.png'),
        '-c:v', 'libx264',
        '-profile:v', 'high',
        '-crf', '20',
        '-pix_fmt', 'yuv420p',
        output_video_path
    ]
    subprocess.run(command, check=True)

def create_art0_video(background_path, source_directory, outro_image_path, num_sets, fade_frames=100):
    categories = ["pink", "green", "cyan", "red", "yellow", "orange", "blue", "indigo"]
    base_name = os.path.basename(os.path.normpath(source_directory))
    short_uid = uuid.uuid4().hex[:8]
    frames_directory = f'/tmp/.tmp_frames_{short_uid}'
    os.makedirs(frames_directory, exist_ok=True)

    set_image_paths = sample_unique_image_sets(source_directory, categories, num_sets)
    total_frames = create_sequential_fade_animation_with_background(background_path, set_image_paths, frames_directory, fade_frames)
    
    add_outro_frames(frames_directory, outro_image_path, total_frames, fade_frames)
    
    results_directory = "./results/MacStudio"
    output_video_path = os.path.join(results_directory, f"{base_name}-{num_sets}sets-{short_uid}.mp4")
    
    create_video_from_frames(frames_directory, output_video_path, 60)
    print(f"Video created successfully: {output_video_path}")
    shutil.rmtree(frames_directory)

if __name__ == "__main__":
    num_sets = 1
    source_directory = "./assets/zones/vegas/v0/3360"
    background_path = "./assets/intro/intro@3360.png"
    outro_image_path = "./assets/outro/outro-vegas@3360.png"
    create_art0_video(background_path, source_directory, outro_image_path, num_sets)


In [ ]:
# Working

In [ ]:
import os
import random
import subprocess
import shutil
from collections import defaultdict
from PIL import Image
import uuid
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm  # Import tqdm for the progress bar

def sample_unique_image_sets(source_directory, categories, num_sets):
    images_by_category = defaultdict(list)
    for filename in os.listdir(source_directory):
        for category in categories:
            if category in filename:
                images_by_category[category].append(os.path.join(source_directory, filename))

    for category, images in images_by_category.items():
        if len(images) < num_sets:
            raise ValueError(f"Not enough images for category '{category}'. Required: {num_sets}, Available: {len(images)}")

    preselected_images_by_category = {category: random.sample(images, num_sets) for category, images in images_by_category.items()}

    set_image_paths = []
    for i in range(num_sets):
        for category in categories:
            selected_image = preselected_images_by_category[category][i]
            set_image_paths.append(selected_image)

    return set_image_paths

def create_frame(args):
    background_image, frame_num, total_frames, fade_frames, images_with_masks, frames_directory = args
    current_frame = background_image.copy()
    current_image_index = (frame_num - 1) // fade_frames
    current_image_phase = (frame_num - 1) % fade_frames

    for i, (image, mask) in enumerate(images_with_masks):
        if i <= current_image_index:
            opacity = int(255 * current_image_phase / fade_frames) if i == current_image_index else 255
            fading_mask = mask.point(lambda p: min(p, opacity))
            fading_image = image.copy()
            fading_image.putalpha(fading_mask)
            current_frame = Image.alpha_composite(current_frame, fading_image)

    frame_path = os.path.join(frames_directory, f"frame_{frame_num:05}.png")
    current_frame.save(frame_path)

def create_sequential_fade_animation_with_background(background_path, image_paths, frames_directory, fade_frames=100):
    background_image = Image.open(background_path).convert('RGBA')
    images_with_masks = [(Image.open(path).convert('RGBA'), Image.open(path).convert('RGBA').split()[3]) for path in image_paths]
    total_images = len(images_with_masks)
    total_frames = fade_frames * total_images

    args_list = [(background_image, frame_num, total_frames, fade_frames, images_with_masks, frames_directory) for frame_num in range(1, total_frames + 1)]
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(create_frame, args_list), total=total_frames, desc="Generating animation frames"))

    return total_frames

def add_outro_frames(frames_directory, outro_image_path, start_frame_num, fade_frames=300):
    outro_image = Image.open(outro_image_path).convert("RGBA")
    for i in tqdm(range(1, fade_frames + 1), desc="Adding outro frames"):
        opacity = int(255 * (i / fade_frames))
        outro_frame = Image.new("RGBA", outro_image.size, (255, 255, 255, 0))
        outro_frame.paste(outro_image, (0, 0), outro_image.split()[3].point(lambda _: opacity))
        
        frame_path = os.path.join(frames_directory, f"frame_{start_frame_num + i:05}.png")
        outro_frame.save(frame_path)

def create_video_from_frames(frames_directory, output_video_path, framerate=60):
    command = [
        'ffmpeg',
        '-framerate', str(framerate),
        '-i', os.path.join(frames_directory, 'frame_%05d.png'),
        '-c:v', 'libx264',
        '-profile:v', 'high',
        '-crf', '20',
        '-pix_fmt', 'yuv420p',
        output_video_path
    ]
    subprocess.run(command, check=True)

def create_art0_video(background_path, source_directory, outro_image_path, num_sets, fade_frames=100, results_directory=None):
    categories = ["pink", "green", "cyan", "red", "yellow", "orange", "blue", "indigo"]
    base_name = os.path.basename(os.path.normpath(source_directory))
    short_uid = uuid.uuid4().hex[:8]
    frames_directory = f'/tmp/.tmp_frames_{short_uid}'
    os.makedirs(frames_directory, exist_ok=True)

    set_image_paths = sample_unique_image_sets(source_directory, categories, num_sets)
    total_frames = create_sequential_fade_animation_with_background(background_path, set_image_paths, frames_directory, fade_frames)
    
    add_outro_frames(frames_directory, outro_image_path, total_frames, fade_frames)
    
    if results_directory is None:
        results_directory = "/Users/laeh/Desktop/art0/results/videos/v1+outro/vegas"
    os.makedirs(results_directory, exist_ok=True)  # Ensure the results directory exists
    output_video_path = os.path.join(results_directory, f"{base_name}-{num_sets}sets-{short_uid}.mp4")
    
    create_video_from_frames(frames_directory, output_video_path, 60)
    print(f"Video created successfully: {output_video_path}")
    shutil.rmtree(frames_directory)


if __name__ == "__main__":
    for _ in range(1):  # This loop will repeat the contained code 10 times

        num_sets = 3
        source_directory = "/Users/laeh/Desktop/art0/assets/zones/light/v2/1680"
        background_path = "/Users/laeh/Desktop/art0/assets/intro/intro@1680.png"
        outro_image_path = "/Users/laeh/Desktop/art0/assets/outro/outro-light@1680.png"
        results_directory = "/Users/laeh/Desktop/art0/results/videos/light-3s"
        create_art0_video(background_path, source_directory, outro_image_path, num_sets, results_directory=results_directory)

# Interesting, just one shape at a time.

In [ ]:
import os
import random
import shutil
import subprocess
from collections import defaultdict
from PIL import Image
import uuid
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def sample_unique_image_sets(source_directory, categories, num_sets):
    images_by_category = defaultdict(list)
    for filename in os.listdir(source_directory):
        for category in categories:
            if category in filename:
                images_by_category[category].append(os.path.join(source_directory, filename))

    for category, images in images_by_category.items():
        if len(images) < num_sets:
            raise ValueError(f"Not enough images for category '{category}'. Required: {num_sets}, Available: {len(images)}")

    preselected_images_by_category = {category: random.sample(images, num_sets) for category, images in images_by_category.items()}

    set_image_paths = []
    for i in range(num_sets):
        for category in categories:
            selected_image = preselected_images_by_category[category][i]
            set_image_paths.append(selected_image)

    return set_image_paths

def create_frame(args):
    background_image, frame_num, total_frames, fade_frames, intermediate_frames, images_with_masks, frames_directory = args
    cycle_length = fade_frames + intermediate_frames  # Total length of one image's display cycle including the fade and static display
    total_cycles = len(images_with_masks)
    
    # Determine the current cycle and phase within the cycle
    current_cycle = (frame_num - 1) // cycle_length
    phase_within_cycle = (frame_num - 1) % cycle_length
    fade_in_phase = phase_within_cycle < fade_frames

    # Start with a clean background for each frame
    current_frame = background_image.copy()
    
    for i in range(current_cycle + 1):
        image, mask = images_with_masks[i]
        if i < current_cycle:  # For previous cycles, images should be fully opaque
            opacity = 255
        elif fade_in_phase:  # Current image fading in
            opacity = int((phase_within_cycle / fade_frames) * 255)
        else:  # Current image fully visible
            opacity = 255
        
        # Apply opacity to the mask
        fading_mask = mask.point(lambda p: min(p, opacity))
        fading_image = image.copy()
        fading_image.putalpha(fading_mask)
        current_frame = Image.alpha_composite(current_frame, fading_image)

    frame_path = os.path.join(frames_directory, f"frame_{frame_num:05}.png")
    current_frame.save(frame_path)

def create_sequential_fade_animation_with_background(background_path, image_paths, frames_directory, fade_frames=100, intermediate_frames=120):
    background_image = Image.open(background_path).convert('RGBA')
    images_with_masks = [(Image.open(path).convert('RGBA'), Image.open(path).convert('L')) for path in image_paths]
    total_images = len(images_with_masks)
    total_frames = (fade_frames + intermediate_frames) * total_images - intermediate_frames  # Adjust for final set of intermediate frames

    args_list = [(background_image, frame_num, total_frames, fade_frames, intermediate_frames, images_with_masks, frames_directory) for frame_num in range(1, total_frames + 1)]
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(create_frame, args_list), total=total_frames, desc="Generating animation frames"))

    return total_frames

# Remaining functions (add_outro_frames, create_video_from_frames, create_art0_video) do not require modifications for this change.


def add_outro_frames(frames_directory, outro_image_path, start_frame_num, fade_frames=300):
    outro_image = Image.open(outro_image_path).convert("RGBA")
    for i in tqdm(range(1, fade_frames + 1), desc="Adding outro frames"):
        opacity = int(255 * (i / fade_frames))
        outro_frame = Image.new("RGBA", outro_image.size, (255, 255, 255, 0))
        outro_frame.paste(outro_image, (0, 0), outro_image.split()[3].point(lambda _: opacity))
        
        frame_path = os.path.join(frames_directory, f"frame_{start_frame_num + i:05}.png")
        outro_frame.save(frame_path)

def create_video_from_frames(frames_directory, output_video_path, framerate=60):
    command = [
        'ffmpeg',
        '-framerate', str(framerate),
        '-i', os.path.join(frames_directory, 'frame_%05d.png'),
        '-c:v', 'libx264',
        '-profile:v', 'high',
        '-crf', '20',
        '-pix_fmt', 'yuv420p',
        output_video_path
    ]
    subprocess.run(command, check=True)

def create_art0_video(background_path, source_directory, outro_image_path, num_sets, fade_frames=100, results_directory=None):
    categories = ["pink", "green", "cyan", "red", "yellow", "orange", "blue", "indigo"]
    base_name = os.path.basename(os.path.normpath(source_directory))
    short_uid = uuid.uuid4().hex[:8]
    frames_directory = f'/tmp/.tmp_frames_{short_uid}'
    os.makedirs(frames_directory, exist_ok=True)

    set_image_paths = sample_unique_image_sets(source_directory, categories, num_sets)
    total_frames = create_sequential_fade_animation_with_background(background_path, set_image_paths, frames_directory, fade_frames)
    
    add_outro_frames(frames_directory, outro_image_path, total_frames, fade_frames)
    
    if results_directory is None:
        results_directory = "/Users/laeh/Desktop/art0/results/videos/v1+outro/vegas"
    os.makedirs(results_directory, exist_ok=True)  # Ensure the results directory exists
    output_video_path = os.path.join(results_directory, f"{base_name}-{num_sets}sets-{short_uid}.mp4")
    
    create_video_from_frames(frames_directory, output_video_path, 60)
    print(f"Video created successfully: {output_video_path}")
    shutil.rmtree(frames_directory)


if __name__ == "__main__":
    for _ in range(1):  # This loop will repeat the contained code 10 times

        num_sets = 2
        source_directory = "/Users/laeh/Desktop/art0/assets/zones/louvre/v2/1680"
        background_path = "/Users/laeh/Desktop/art0/assets/intro/intro@1680.png"
        outro_image_path = "/Users/laeh/Desktop/art0/assets/outro/outro-louvre@1680.png"
        results_directory = "/Users/laeh/Desktop/art0/results/videos/test-1s"
        create_art0_video(background_path, source_directory, outro_image_path, num_sets, results_directory=results_directory)

In [ ]:
import os
import random
import shutil
import subprocess
from collections import defaultdict
from PIL import Image
import uuid
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def sample_unique_image_sets(source_directory, categories, num_sets):
    images_by_category = defaultdict(list)
    for filename in os.listdir(source_directory):
        for category in categories:
            if category in filename:
                images_by_category[category].append(os.path.join(source_directory, filename))

    for category, images in images_by_category.items():
        if len(images) < num_sets:
            raise ValueError(f"Not enough images for category '{category}'. Required: {num_sets}, Available: {len(images)}")

    preselected_images_by_category = {category: random.sample(images, num_sets) for category, images in images_by_category.items()}

    set_image_paths = []
    for i in range(num_sets):
        for category in categories:
            selected_image = preselected_images_by_category[category][i]
            set_image_paths.append(selected_image)

    return set_image_paths

def create_frame(args):
    background_image, frame_num, total_frames, fade_frames, intermediate_frames, images_with_masks, frames_directory = args
    cycle_length = fade_frames + intermediate_frames  # Total length of one image's display cycle
    total_cycles = len(images_with_masks)
    
    # Calculate the current cycle (image being processed) and the phase within the cycle
    current_cycle = (frame_num - 1) // cycle_length
    phase_within_cycle = (frame_num - 1) % cycle_length
    is_fading_phase = phase_within_cycle < fade_frames

    # Start with a fresh copy of the background for each frame
    current_frame = background_image.copy()

    for i, (image, mask) in enumerate(images_with_masks):
        if i < current_cycle:
            # Past images should be fully visible
            opacity = 255
        elif i == current_cycle:
            # Current image could be in the process of fading in
            if is_fading_phase:
                opacity = int((phase_within_cycle / float(fade_frames)) * 255)
            else:
                # If not in fading phase, it means the image is fully visible
                opacity = 255
        else:
            # Future images should not be processed yet
            continue

        # Adjust the image's mask based on the current opacity
        adjusted_mask = mask.point(lambda p: int(p * opacity / 255))

        # Prepare the image for compositing
        fading_image = image.copy()
        fading_image.putalpha(adjusted_mask)

        # Composite the image onto the current frame
        current_frame.paste(fading_image, (0, 0), fading_image)

    # Save the current frame
    frame_path = os.path.join(frames_directory, f"frame_{frame_num:05}.png")
    current_frame.save(frame_path)


def create_sequential_fade_animation_with_background(background_path, image_paths, frames_directory, fade_frames=100, intermediate_frames=120):
    background_image = Image.open(background_path).convert('RGBA')
    images_with_masks = [(Image.open(path).convert('RGBA'), Image.open(path).convert('L')) for path in image_paths]
    total_images = len(images_with_masks)
    total_frames = (fade_frames + intermediate_frames) * total_images - intermediate_frames  # Adjust for final set of intermediate frames

    args_list = [(background_image, frame_num, total_frames, fade_frames, intermediate_frames, images_with_masks, frames_directory) for frame_num in range(1, total_frames + 1)]
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(create_frame, args_list), total=total_frames, desc="Generating animation frames"))

    return total_frames

# Remaining functions (add_outro_frames, create_video_from_frames, create_art0_video) do not require modifications for this change.


def add_outro_frames(frames_directory, outro_image_path, start_frame_num, fade_frames=300):
    outro_image = Image.open(outro_image_path).convert("RGBA")
    for i in tqdm(range(1, fade_frames + 1), desc="Adding outro frames"):
        opacity = int(255 * (i / fade_frames))
        outro_frame = Image.new("RGBA", outro_image.size, (255, 255, 255, 0))
        outro_frame.paste(outro_image, (0, 0), outro_image.split()[3].point(lambda _: opacity))
        
        frame_path = os.path.join(frames_directory, f"frame_{start_frame_num + i:05}.png")
        outro_frame.save(frame_path)

def create_video_from_frames(frames_directory, output_video_path, framerate=60):
    command = [
        'ffmpeg',
        '-framerate', str(framerate),
        '-i', os.path.join(frames_directory, 'frame_%05d.png'),
        '-c:v', 'libx264',
        '-profile:v', 'high',
        '-crf', '20',
        '-pix_fmt', 'yuv420p',
        output_video_path
    ]
    subprocess.run(command, check=True)

def create_art0_video(background_path, source_directory, outro_image_path, num_sets, fade_frames=100, results_directory=None):
    categories = ["pink", "green", "cyan", "red", "yellow", "orange", "blue", "indigo"]
    base_name = os.path.basename(os.path.normpath(source_directory))
    short_uid = uuid.uuid4().hex[:8]
    frames_directory = f'/tmp/.tmp_frames_{short_uid}'
    os.makedirs(frames_directory, exist_ok=True)

    set_image_paths = sample_unique_image_sets(source_directory, categories, num_sets)
    total_frames = create_sequential_fade_animation_with_background(background_path, set_image_paths, frames_directory, fade_frames)
    
    add_outro_frames(frames_directory, outro_image_path, total_frames, fade_frames)
    
    if results_directory is None:
        results_directory = "/Users/laeh/Desktop/art0/results/videos/v1+outro/vegas"
    os.makedirs(results_directory, exist_ok=True)  # Ensure the results directory exists
    output_video_path = os.path.join(results_directory, f"{base_name}-{num_sets}sets-{short_uid}.mp4")
    
    create_video_from_frames(frames_directory, output_video_path, 60)
    print(f"Video created successfully: {output_video_path}")
    shutil.rmtree(frames_directory)


if __name__ == "__main__":
    for _ in range(1):  # This loop will repeat the contained code 10 times

        num_sets = 2
        source_directory = "/Users/laeh/Desktop/art0/assets/zones/louvre/v2/1680"
        background_path = "/Users/laeh/Desktop/art0/assets/intro/intro@1680.png"
        outro_image_path = "/Users/laeh/Desktop/art0/assets/outro/outro-louvre@1680.png"
        results_directory = "/Users/laeh/Desktop/art0/results/videos/test-1s"
        create_art0_video(background_path, source_directory, outro_image_path, num_sets, results_directory=results_directory)

In [ ]:
import os
import random
import subprocess
import shutil
from collections import defaultdict
from PIL import Image
import uuid
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def sample_unique_image_sets(source_directory, categories, num_sets):
    images_by_category = defaultdict(list)
    for filename in os.listdir(source_directory):
        for category in categories:
            if category in filename:
                images_by_category[category].append(os.path.join(source_directory, filename))

    for category, images in images_by_category.items():
        if len(images) < num_sets:
            raise ValueError(f"Not enough images for category '{category}'. Required: {num_sets}, Available: {len(images)}")

    preselected_images_by_category = {category: random.sample(images, num_sets) for category, images in images_by_category.items()}

    set_image_paths = []
    for i in range(num_sets):
        for category in categories:
            selected_image = preselected_images_by_category[category][i]
            set_image_paths.append(selected_image)

    return set_image_paths

def create_frame(args):
    background_image, frame_num, total_frames, fade_frames, still_frames, images_with_masks, frames_directory = args
    cycle_length = fade_frames + still_frames
    current_image_index = (frame_num - 1) // cycle_length
    current_phase_in_cycle = (frame_num - 1) % cycle_length
    current_frame = background_image.copy()

    for i, (image, mask) in enumerate(images_with_masks):
        if i < current_image_index:
            opacity = 255
        elif i == current_image_index:
            if current_phase_in_cycle < fade_frames:
                opacity = int(255 * current_phase_in_cycle / fade_frames)
            else:
                opacity = 255
        else:
            break

        fading_mask = mask.point(lambda p: min(p, opacity))
        fading_image = image.copy()
        fading_image.putalpha(fading_mask)
        current_frame = Image.alpha_composite(current_frame, fading_image)

    frame_path = os.path.join(frames_directory, f"frame_{frame_num:05}.png")
    current_frame.save(frame_path)

def create_sequential_fade_animation_with_background(background_path, image_paths, frames_directory, fade_frames=100, still_frames=0):
    background_image = Image.open(background_path).convert('RGBA')
    images_with_masks = [(Image.open(path).convert('RGBA'), Image.open(path).convert('RGBA').split()[3]) for path in image_paths]
    total_images = len(images_with_masks)
    total_frames = (fade_frames + still_frames) * total_images - still_frames

    args_list = [(background_image, frame_num, total_frames, fade_frames, still_frames, images_with_masks, frames_directory) for frame_num in range(1, total_frames + 1)]
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(create_frame, args_list), total=total_frames, desc="Generating animation frames"))

    return total_frames

def add_outro_frames(frames_directory, outro_image_path, start_frame_num, fade_frames=300):
    outro_image = Image.open(outro_image_path).convert("RGBA")
    for i in tqdm(range(1, fade_frames + 1), desc="Adding outro frames"):
        opacity = int(255 * (i / fade_frames))
        outro_frame = Image.new("RGBA", outro_image.size, (255, 255, 255, 0))
        outro_frame.paste(outro_image, (0, 0), outro_image.split()[3].point(lambda _: opacity))
        
        frame_path = os.path.join(frames_directory, f"frame_{start_frame_num + i:05}.png")
        outro_frame.save(frame_path)

def create_video_from_frames(frames_directory, output_video_path, framerate=60):
    command = [
        'ffmpeg',
        '-framerate', str(framerate),
        '-i', os.path.join(frames_directory, 'frame_%05d.png'),
        '-c:v', 'libx264',
        '-profile:v', 'high',
        '-crf', '20',
        '-pix_fmt', 'yuv420p',
        output_video_path
    ]
    subprocess.run(command, check=True)

def create_art0_video(background_path, source_directory, outro_image_path, num_sets, fade_frames=100, still_frames=100, results_directory=None):
    categories = ["pink", "green", "cyan", "red", "yellow", "orange", "blue", "indigo"]
    base_name = os.path.basename(os.path.normpath(source_directory))
    short_uid = uuid.uuid4().hex[:8]
    frames_directory = f'/tmp/.tmp_frames_{short_uid}'
    os.makedirs(frames_directory, exist_ok=True)

    set_image_paths = sample_unique_image_sets(source_directory, categories, num_sets)
    total_frames = create_sequential_fade_animation_with_background(background_path, set_image_paths, frames_directory, fade_frames, still_frames)
    
    add_outro_frames(frames_directory, outro_image_path, total_frames, fade_frames)
    
    if results_directory is None:
        results_directory = "/path/to/results/directory"  # Update this path as necessary
    os.makedirs(results_directory, exist_ok=True)
    output_video_path = os.path.join(results_directory, f"{base_name}-{num_sets}sets-{short_uid}.mp4")
    
    create_video_from_frames(frames_directory, output_video_path, 60)
    print(f"Video created successfully: {output_video_path}")
    shutil.rmtree(frames_directory)

if __name__ == "__main__":
    num_sets = 3  # Example: Adjust as needed
    source_directory = "/Users/laeh/Desktop/art0/assets/zones/light/v2/420"
    background_path = "/Users/laeh/Desktop/art0/assets/intro/intro@420.png"
    outro_image_path = "/Users/laeh/Desktop/art0/assets/outro/outro-light@420.png"
    results_directory = "/Users/laeh/Desktop/art0/results/videos/test-1s"
    create_art0_video(background_path, source_directory, outro_image_path, num_sets, results_directory=results_directory)


# Working with outro 

### Working with outro (Slow)

In [ ]:
import os
import random
import subprocess
import shutil
from collections import defaultdict
from PIL import Image
import uuid
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def sample_unique_image_sets(source_directory, categories, num_sets):
    images_by_category = defaultdict(list)
    for filename in os.listdir(source_directory):
        for category in categories:
            if category in filename:
                images_by_category[category].append(os.path.join(source_directory, filename))

    for category, images in images_by_category.items():
        if len(images) < num_sets:
            raise ValueError(f"Not enough images for category '{category}'. Required: {num_sets}, Available: {len(images)}")

    preselected_images_by_category = {category: random.sample(images, num_sets) for category, images in images_by_category.items()}

    set_image_paths = []
    for i in range(num_sets):
        for category in categories:
            selected_image = preselected_images_by_category[category][i]
            set_image_paths.append(selected_image)

    return set_image_paths

def create_frame(args):
    background_image, frame_num, total_frames, fade_frames, still_frames, images_with_masks, frames_directory = args
    cycle_length = fade_frames + still_frames
    current_image_index = (frame_num - 1) // cycle_length
    current_phase_in_cycle = (frame_num - 1) % cycle_length
    current_frame = background_image.copy()

    for i, (image, mask) in enumerate(images_with_masks):
        if i < current_image_index:
            opacity = 255
        elif i == current_image_index:
            if current_phase_in_cycle < fade_frames:
                opacity = int(255 * current_phase_in_cycle / fade_frames)
            else:
                opacity = 255
        else:
            break

        fading_mask = mask.point(lambda p: min(p, opacity))
        fading_image = image.copy()
        fading_image.putalpha(fading_mask)
        current_frame = Image.alpha_composite(current_frame, fading_image)

    frame_path = os.path.join(frames_directory, f"frame_{frame_num:05}.png")
    current_frame.save(frame_path)

def create_sequential_fade_animation_with_background(background_path, image_paths, frames_directory, fade_frames=100, still_frames=0):
    background_image = Image.open(background_path).convert('RGBA')
    images_with_masks = [(Image.open(path).convert('RGBA'), Image.open(path).convert('RGBA').split()[3]) for path in image_paths]
    total_images = len(images_with_masks)
    total_frames = (fade_frames + still_frames) * total_images + still_frames  # Corrected to add still_frames after the last image

    args_list = [(background_image, frame_num, total_frames, fade_frames, still_frames, images_with_masks, frames_directory) for frame_num in range(1, total_frames + 1)]
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(create_frame, args_list), total=total_frames, desc="Generating animation frames"))

    return total_frames

def add_outro_frames(frames_directory, outro_image_path, start_frame_num, fade_frames=300):
    outro_image = Image.open(outro_image_path).convert("RGBA")
    for i in tqdm(range(1, fade_frames + 1), desc="Adding outro frames"):
        opacity = int(255 * (i / fade_frames))
        outro_frame = Image.new("RGBA", outro_image.size, (255, 255, 255, 0))
        outro_frame.paste(outro_image, (0, 0), outro_image.split()[3].point(lambda _: opacity))
        
        frame_path = os.path.join(frames_directory, f"frame_{start_frame_num + i - 1:05}.png")  # Ensure correct indexing
        outro_frame.save(frame_path)

def create_video_from_frames(frames_directory, output_video_path, framerate=60):
    command = [
        'ffmpeg',
        '-framerate', str(framerate),
        '-i', os.path.join(frames_directory, 'frame_%05d.png'),
        '-c:v', 'libx264',
        '-profile:v', 'high',
        '-crf', '20',
        '-pix_fmt', 'yuv420p',
        output_video_path
    ]
    subprocess.run(command, check=True)

def create_art0_video(background_path, source_directory, outro_image_path, num_sets, fade_frames=100, still_frames=100, results_directory=None):
    categories = ["pink", "green", "cyan", "red", "yellow", "orange", "blue", "indigo"]
    base_name = os.path.basename(os.path.normpath(source_directory))
    short_uid = uuid.uuid4().hex[:8]
    frames_directory = f'/tmp/.tmp_frames_{short_uid}'
    os.makedirs(frames_directory, exist_ok=True)

    set_image_paths = sample_unique_image_sets(source_directory, categories, num_sets)
    total_frames = create_sequential_fade_animation_with_background(background_path, set_image_paths, frames_directory, fade_frames, still_frames)
    
    # Calculate the correct start frame for the outro
    add_outro_frames(frames_directory, outro_image_path, total_frames + 1, fade_frames)  # Corrected to start outro correctly
    
    if results_directory is None:
        results_directory = "/path/to/results/directory"  # Update this path as necessary
    os.makedirs(results_directory, exist_ok=True)
    output_video_path = os.path.join(results_directory, f"{base_name}-{num_sets}sets-{short_uid}.mp4")
    
    create_video_from_frames(frames_directory, output_video_path, 60)
    print(f"Video created successfully: {output_video_path}")
    shutil.rmtree(frames_directory)

if __name__ == "__main__":

    for _ in range(1):  # This loop will repeat the contained code 10 times
        num_sets = 1
        source_directory = "./assets/zones/light/v0/1680"
        background_path = "./assets/intro/intro@1680.png"
        outro_image_path = "./assets/outro/outro-light@1680.png"
        results_directory = "/Users/laeh/Desktop/art0-results-new"
        create_art0_video(background_path, source_directory, outro_image_path, num_sets, results_directory=results_directory)



### With Outro (Multithreading)

In [ ]:
import os
import random
import subprocess
import shutil
from collections import defaultdict
from PIL import Image
import uuid
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def sample_unique_image_sets(source_directory, categories, num_sets):
    images_by_category = defaultdict(list)
    for filename in os.listdir(source_directory):
        for category in categories:
            if category in filename:
                images_by_category[category].append(os.path.join(source_directory, filename))

    for category, images in images_by_category.items():
        if len(images) < num_sets:
            raise ValueError(f"Not enough images for category '{category}'. Required: {num_sets}, Available: {len(images)}")

    preselected_images_by_category = {category: random.sample(images, num_sets) for category, images in images_by_category.items()}

    set_image_paths = []
    for i in range(num_sets):
        for category in categories:
            selected_image = preselected_images_by_category[category][i]
            set_image_paths.append(selected_image)

    return set_image_paths

def create_frame(args):
    background_image, frame_num, total_frames, fade_frames, still_frames, images_with_masks, frames_directory, frame_background_path = args
    cycle_length = fade_frames + still_frames
    current_image_index = (frame_num - 1) // cycle_length
    current_phase_in_cycle = (frame_num - 1) % cycle_length
    current_frame = background_image.copy()

    for i, (image, mask) in enumerate(images_with_masks):
        if i < current_image_index:
            opacity = 255
        elif i == current_image_index:
            if current_phase_in_cycle < fade_frames:
                opacity = int(255 * current_phase_in_cycle / fade_frames)
            else:
                opacity = 255
        else:
            break

        fading_mask = mask.point(lambda p: min(p, opacity))
        fading_image = image.copy()
        fading_image.putalpha(fading_mask)
        current_frame = Image.alpha_composite(current_frame, fading_image)

    # Adjust the size of the current_frame to match the frame background, if necessary
    frame_overlay = Image.open(frame_background_path).convert('RGBA')
    bg_width, bg_height = frame_overlay.size

    # Create a transparent image the size of the frame background
    transparent_canvas = Image.new("RGBA", (bg_width, bg_height), (0, 0, 0, 0))

    # Calculate position to center the current_frame on the transparent canvas
    fg_width, fg_height = current_frame.size
    x_offset = (bg_width - fg_width) // 2
    y_offset = (bg_height - fg_height) // 2

    # Paste the current frame onto the transparent canvas at the calculated position
    transparent_canvas.paste(current_frame, (x_offset, y_offset), current_frame)

    # Composite the frame overlay on top of the centered current frame
    final_composite = Image.alpha_composite(transparent_canvas, frame_overlay)

    frame_path = os.path.join(frames_directory, f"frame_{frame_num:05}.png")
    final_composite.save(frame_path)


def create_sequential_fade_animation_with_background(background_path, image_paths, frames_directory, fade_frames, still_frames, frame_background_path):
    background_image = Image.open(background_path).convert('RGBA')
    images_with_masks = [(Image.open(path).convert('RGBA'), Image.open(path).convert('RGBA').split()[3]) for path in image_paths]
    total_images = len(images_with_masks)
    total_frames = (fade_frames + still_frames) * total_images + still_frames

    args_list = [(background_image, frame_num, total_frames, fade_frames, still_frames, images_with_masks, frames_directory, frame_background_path) for frame_num in range(1, total_frames + 1)]
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(create_frame, args_list), total=total_frames, desc="Generating animation frames"))

    return total_frames

def add_outro_frames(frames_directory, outro_image_path, frame_background_path, start_frame_num, fade_frames=300):
    outro_image = Image.open(outro_image_path).convert("RGBA")
    frame_bg_image = Image.open(frame_background_path).convert('RGBA')
    bg_width, bg_height = frame_bg_image.size

    # Create a transparent image the size of the frame background
    transparent_canvas = Image.new("RGBA", (bg_width, bg_height), (0, 0, 0, 0))

    # Calculate position to center the outro image on the transparent canvas
    outro_width, outro_height = outro_image.size
    x_offset = (bg_width - outro_width) // 2
    y_offset = (bg_height - outro_height) // 2

    for i in tqdm(range(1, fade_frames + 1), desc="Adding outro frames"):
        opacity = int(255 * (i / fade_frames))
        outro_frame_with_opacity = outro_image.copy()
        outro_frame_with_opacity.putalpha(opacity)

        # Paste the outro image onto the transparent canvas at the calculated position
        temp_canvas = transparent_canvas.copy()
        temp_canvas.paste(outro_frame_with_opacity, (x_offset, y_offset), outro_frame_with_opacity)

        # Now composite the frame background over the temp canvas
        final_frame = Image.alpha_composite(temp_canvas, frame_bg_image)

        frame_path = os.path.join(frames_directory, f"frame_{start_frame_num + i - 1:05}.png")
        final_frame.save(frame_path)


def create_video_from_frames(frames_directory, output_video_path, framerate=60):
    command = [
        'ffmpeg',
        '-framerate', str(framerate),
        '-i', os.path.join(frames_directory, 'frame_%05d.png'),
        '-c:v', 'libx264',
        '-profile:v', 'high',
        '-crf', '20',
        '-pix_fmt', 'yuv420p',
        output_video_path
    ]
    subprocess.run(command, check=True)

def create_art0_video(background_path, source_directory, outro_image_path, num_sets, fade_frames, still_frames, frame_background, results_directory=None):
    categories = ["pink", "green", "cyan", "red", "yellow", "orange", "blue", "indigo"]
    base_name = os.path.basename(os.path.normpath(source_directory))
    short_uid = uuid.uuid4().hex[:8]
    frames_directory = f'/tmp/.tmp_frames_{short_uid}'
    os.makedirs(frames_directory, exist_ok=True)

    set_image_paths = sample_unique_image_sets(source_directory, categories, num_sets)
    total_frames = create_sequential_fade_animation_with_background(background_path, set_image_paths, frames_directory, fade_frames, still_frames, frame_background)
    
    add_outro_frames(frames_directory, outro_image_path, frame_background, total_frames + 1, fade_frames)
    
    if results_directory is None:
        results_directory = "/path/to/results/directory"
    os.makedirs(results_directory, exist_ok=True)
    output_video_path = os.path.join(results_directory, f"{base_name}-{num_sets}sets-{short_uid}.mp4")
    
    create_video_from_frames(frames_directory, output_video_path, 60)
    print(f"Video created successfully: {output_video_path}")
    shutil.rmtree(frames_directory)

if __name__ == "__main__":
    for _ in range(10):  # This loop will repeat the contained code 10 times
        num_sets = 1
        source_directory = "./assets/zones/vegas/v0/1680"
        background_path = "./assets/intro/intro@1680.png"
        outro_image_path = "./assets/outro/outro-vegas@1680.png"
        results_directory = "/Users/laeh/Desktop/art0/results/NEW"
        frame_background = "/Users/laeh/Desktop/art0/assets/cadre/5l-inverted.png"  # Path to the frame background image
        create_art0_video(background_path, source_directory, outro_image_path, num_sets, 100, 100, frame_background, results_directory)

        # num_sets = 12
        # source_directory = "./assets/zones/vegas/v0/1680"
        # background_path = "./assets/intro/intro@1680.png"
        # outro_image_path = "./assets/outro/outro-vegas@1680.png"
        # results_directory = "/Users/laeh/Desktop/art0/results/NEW"
        # frame_background = "/Users/laeh/Desktop/art0/assets/cadre/5l.png"  # Path to the frame background image
        # create_art0_video(background_path, source_directory, outro_image_path, num_sets, 100, 100, frame_background, results_directory)


### Working with outro (FasterTest)

In [ ]:
import os
import random
import subprocess
import shutil
from collections import defaultdict
from PIL import Image
import uuid
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def sample_unique_image_sets(source_directory, categories, num_sets):
    images_by_category = defaultdict(list)
    for filename in os.listdir(source_directory):
        for category in categories:
            if category in filename:
                images_by_category[category].append(os.path.join(source_directory, filename))

    for category, images in images_by_category.items():
        if len(images) < num_sets:
            raise ValueError(f"Not enough images for category '{category}'. Required: {num_sets}, Available: {len(images)}")

    preselected_images_by_category = {category: random.sample(images, num_sets) for category, images in images_by_category.items()}

    set_image_paths = []
    for i in range(num_sets):
        for category in categories:
            selected_image = preselected_images_by_category[category][i]
            set_image_paths.append(selected_image)

    return set_image_paths

def create_frame_optimized_wrapper(args):
    frame_num, cumulative_backgrounds, current_image, current_mask, fade_frames, cycle_length, frames_directory, cycle_index, is_fading, phase_within_cycle = args
    
    if cycle_index >= len(cumulative_backgrounds) - 1:
        # Past the last image, show the final result
        current_frame = cumulative_backgrounds[-1]
    elif is_fading:
        base_background = cumulative_backgrounds[cycle_index]
        opacity = int((phase_within_cycle / float(fade_frames)) * 255)
        fading_mask = current_mask.point(lambda p: int(p * opacity / 255))
        fading_image = current_image.copy()
        fading_image.putalpha(fading_mask)
        current_frame = Image.alpha_composite(base_background, fading_image)
    else:
        # Fully opaque phase of current cycle -> equivalent to start of next cycle
        current_frame = cumulative_backgrounds[cycle_index + 1]
        
    frame_path = os.path.join(frames_directory, f"frame_{frame_num:05}.png")
    current_frame.save(frame_path)

def create_sequential_fade_animation_optimized(background_path, image_paths, frames_directory, fade_frames=100, still_frames=0):
    # Load initial background
    initial_background = Image.open(background_path).convert('RGBA')
    
    # Load all images and masks
    images_with_masks = [(Image.open(path).convert('RGBA'), Image.open(path).convert('RGBA').split()[3]) for path in image_paths]
    total_images = len(images_with_masks)
    
    # Pre-compute cumulative backgrounds
    cumulative_backgrounds = [initial_background]
    current_bg = initial_background
    
    print("Pre-computing cumulative backgrounds...")
    for img, mask in tqdm(images_with_masks, desc="Pre-computing"):
        next_bg = Image.alpha_composite(current_bg, img)
        cumulative_backgrounds.append(next_bg)
        current_bg = next_bg
        
    total_frames = (fade_frames + still_frames) * total_images + still_frames
    cycle_length = fade_frames + still_frames

    args_list = []
    for frame_num in range(1, total_frames + 1):
        cycle_index = (frame_num - 1) // cycle_length
        phase_within_cycle = (frame_num - 1) % cycle_length
        
        is_fading = phase_within_cycle < fade_frames
        
        if cycle_index < total_images:
            current_image, current_mask = images_with_masks[cycle_index]
            args = (frame_num, cumulative_backgrounds, current_image, current_mask, fade_frames, cycle_length, frames_directory, cycle_index, is_fading, phase_within_cycle)
            args_list.append(args)
        else:
            # Extra still frames at the end
            args = (frame_num, cumulative_backgrounds, None, None, fade_frames, cycle_length, frames_directory, cycle_index, False, phase_within_cycle)
            args_list.append(args)

    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(create_frame_optimized_wrapper, args_list), total=len(args_list), desc="Generating animation frames (Optimized)"))

    return total_frames

def create_outro_frame_optimized(args):
    i, fade_frames, outro_image, start_frame_num, frames_directory = args
    opacity = int(255 * (i / fade_frames))
    
    outro_frame = Image.new("RGBA", outro_image.size, (255, 255, 255, 0))
    # Replicate original logic: mask is derived from alpha channel but set to uniform opacity
    mask = outro_image.split()[3].point(lambda _: opacity)
    outro_frame.paste(outro_image, (0, 0), mask)
    
    frame_path = os.path.join(frames_directory, f"frame_{start_frame_num + i - 1:05}.png")
    outro_frame.save(frame_path)

def add_outro_frames_optimized(frames_directory, outro_image_path, start_frame_num, fade_frames=300):
    outro_image = Image.open(outro_image_path).convert("RGBA")
    
    args_list = [(i, fade_frames, outro_image, start_frame_num, frames_directory) for i in range(1, fade_frames + 1)]
    
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(create_outro_frame_optimized, args_list), total=fade_frames, desc="Adding outro frames (Parallel)"))

def create_video_from_frames(frames_directory, output_video_path, framerate=60):
    command = [
        'ffmpeg',
        '-framerate', str(framerate),
        '-i', os.path.join(frames_directory, 'frame_%05d.png'),
        '-c:v', 'libx264',
        '-profile:v', 'high',
        '-crf', '20',
        '-pix_fmt', 'yuv420p',
        output_video_path
    ]
    subprocess.run(command, check=True)

def create_art0_video_optimized(background_path, source_directory, outro_image_path, num_sets, fade_frames=100, still_frames=100, results_directory=None):
    categories = ["pink", "green", "cyan", "red", "yellow", "orange", "blue", "indigo"]
    base_name = os.path.basename(os.path.normpath(source_directory))
    short_uid = uuid.uuid4().hex[:8]
    frames_directory = f'/tmp/.tmp_frames_{short_uid}'
    os.makedirs(frames_directory, exist_ok=True)

    set_image_paths = sample_unique_image_sets(source_directory, categories, num_sets)
    total_frames = create_sequential_fade_animation_optimized(background_path, set_image_paths, frames_directory, fade_frames, still_frames)
    
    add_outro_frames_optimized(frames_directory, outro_image_path, total_frames + 1, fade_frames)
    
    if results_directory is None:
        results_directory = "/Users/laeh/Desktop/art0/results/videos/test-optimized"
    os.makedirs(results_directory, exist_ok=True)
    output_video_path = os.path.join(results_directory, f"{base_name}-{num_sets}sets-{short_uid}.mp4")
    
    create_video_from_frames(frames_directory, output_video_path, 60)
    print(f"Video created successfully: {output_video_path}")
    shutil.rmtree(frames_directory)

if __name__ == "__main__":
    num_sets = 1
    source_directory = "./assets/zones/light/v0/1680"
    background_path = "./assets/intro/intro@1680.png"
    outro_image_path = "./assets/outro/outro-light@1680.png"
    results_directory = "/Users/laeh/Desktop/art0/results/videos/test-optimized"
    
    create_art0_video_optimized(background_path, source_directory, outro_image_path, num_sets, results_directory=results_directory)
